In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

import stripe
stripe.api_key = os.getenv('STRIPE_KEY')

from extract_issue import extract_issue
from extract_issue import extract_issue_analyzer
from ChooseService import analyze_situation_sequential

BOLD = '\033[4m'
GREEN = '\033[92m'
RED = '\033[91m'
BLUE = '\033[94m'
END = '\033[0m'

Running Chroma using direct local API.
loaded in 78 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
Running Chroma using direct local API.
loaded in 0 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [2]:
chat = ChatOpenAI(openai_api_key=key)
gpt4 = ChatOpenAI(openai_api_key=key, model='gpt-4')

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=key)
policies = Chroma(persist_directory='data/policies-db/', embedding_function=embeddings)

Running Chroma using direct local API.
loaded in 119 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [4]:
docs = policies.similarity_search('squirrel issue', 3)
content = [doc.page_content for doc in docs]
content = '\n'.join(content)
print(content)

Our pest control services do not cover bed bugs, any type of wildlife (including squirrels, raccoons, and snakes), or any other animal infestations.
While we strive to provide comprehensive pest control services, we do not cover bed bugs, wildlife, or other animal infestations. Customers with these issues should contact qualified professionals who specialize in bed bug treatments or wildlife removal. Our technicians are available to provide recommendations and assistance with identifying these issues. If you have any questions or concerns regarding bed bugs, wildlife, or other animals, please contact us at 703-683-2000.
Not covered pests for quarterly pest control program: Rats, mice, mosquitoes, American roaches, German roaches, roaches, termites, exterior ticks, squirrels


In [5]:
template = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases. Use a maximum of one emoji per two responses.

You should structure the conversation in the following way. Take the lead in the conversation by asking questions. Make sure you complete all the requirements before moving on to the next stage.

Stage 1: Take lead of conversation
Ask the customer's permission to ask questions to understand their issue so you can provide an accurate quote.
Compete when: the customer has given permission

Stage 2: Discover customer issue
Gather details about the customer's issue. Show empathy and try to expose things that are bothering the lead. Be patient and empathetic. 
Complete when: you have the following info:
-The location of the issue on the property
-Property square footage
-Property acreage
-Type of property (single family, townhome, apartment, condo, commercial, etc)
-Name of any pests they are dealing with

Here are some details about your business that may be helpful to answer the question. Do not make up the answer:

{relevant_policies}

Once you have completed these stages, do not continue. Simply write: <end>"""

#save as a prompt
prompt = PromptTemplate(
    input_variables=['relevant_policies'],
    template=template,
)

prompt.save('prompts/mega_prompts/discovery.json')

In [6]:
def format(tag, text):
    return f'{BOLD}{RED}{tag}{END}: {text.content} \n\n'

class Conversation:
    def __init__(self, system_prompt, initial_message=None, show_transcript=True, use_gpt4=False):
        self.system_prompt = system_prompt
        self.initial_message = initial_message
        self.back_and_forth = []
        self.show_transcript = show_transcript
        self.use_gpt4 = use_gpt4
        self.stage = 'discovery'
        self.policies = policies
        self.offers = None
    
    def __repr__(self):
        transcript = ''
        for message in self.history():
            if isinstance(message, AIMessage):
                transcript += format('AI', message)
            elif isinstance(message, HumanMessage):
                transcript += format('Human', message)
            elif isinstance(message, SystemMessage):
                transcript += format('System', message)
        return transcript

    def __str__(self):
        return self.__repr__()
    
    def natural_history(self):
        msgs = []
        for message in self.history():
            if isinstance(message, AIMessage):
                msgs.append("Agent: " + message.content)
            elif isinstance(message, HumanMessage):
                msgs.append("Customer: " + message.content)
        natural_lang = '\n'.join(msgs)
        return natural_lang
    
    def history(self):
        initialization = [
            SystemMessage(content=self.system_prompt.format(relevant_policies=self.policies)), 
            AIMessage(content=self.initial_message)
        ]
        return initialization + self.back_and_forth
    
    def display(self):
        history = self.history()
        last_message = history[-1]
        if self.show_transcript:
            print(self)
        else:
            print(last_message.content)
    
    def update_policies(self, text):
        docs = policies.similarity_search(text, 3)
        content = [doc.page_content for doc in docs]
        content = '\n'.join(content)
        self.policies = content

    def complete(self):
        history = self.history()
        if self.use_gpt4:
            response = gpt4(history)
        else:
            response = chat(history)
        return response

    def toggle_output(self):
        self.show_transcript = not self.show_transcript
        if self.show_transcript:
            print(GREEN + 'Showing transcript', END)
        else:
            print(RED + 'Hiding transcript', END)

    def ask(self, text):
        self.back_and_forth.append(HumanMessage(content=text))
        self.update_policies(text)
        completion = self.complete()
        is_done = '<end>' in completion.content
        
        #handle the conversation script
        if self.stage == 'discovery' and is_done:
            self.back_and_forth.append(AIMessage(content="""Well don't fret John. We see this kind of thing all the time. Give me just a second to check some pricing for you!"""))
            print("Well don't fret John. We see this kind of thing all the time. Give me just a second to check some pricing for you!")
            issues = extract_issue_analyzer(self.natural_history())
            print(issues)
            targets = issues['target_pests']
            history = self.natural_history()
            square_footage = issues['square_footage']
            property_type = issues['property_type']
            acres = issues['acres']

            # run a prebuilt "thought" that retrieves experiences, analyzes the conversation, calculates pricing and then returns a pitch
            pitch = analyze_situation_sequential({
                "property_type": property_type,
                "square_footage": square_footage,
                "acres": acres,
                "target": targets,
                "conversation_history": history
            })
            self.back_and_forth.append(AIMessage(content=pitch['final pitch']))
            self.offers = pitch['service_offers_formatted']
            self.stage = 'free-form'
            
        else:
            self.back_and_forth.append(AIMessage(content=completion.content))

        self.display()

In [27]:
discovery_prompt = load_prompt('prompts/mega_prompts/discovery.json')
initial_message = 'Hi John, how can I help you today?'
agent = Conversation(discovery_prompt, initial_message, use_gpt4=True)

In [28]:
agent.toggle_output()

Hiding transcript 


In [29]:
agent.ask('Do you offer mosquito control?')

Absolutely, we have our Lawnshield Mosquito Strategy that can help you enjoy your yard again! 😊 Can I ask you a few questions to understand your needs and provide you with an accurate quote?


In [30]:
agent.ask('Sure thats fine')

Great, let's start with the location of the issue on your property. Where are you experiencing the most mosquito activity?


In [32]:
agent.ask('3000 square feet')

Thanks! And what about the acreage of your property? This will help us in determining the appropriate service plan for you.


In [33]:
agent.ask('2.5 acres')

Sounds like a nice space! Now, can you please let me know the type of property you have? Is it a single-family home, townhome, apartment, condo, or commercial property?


In [34]:
agent.ask('its a single family town home')

Perfect, thanks for the information. And besides mosquitos, are you dealing with any other pests that you'd like us to address as well? 🐛



In [35]:
agent.ask('sorry I meant its a single family home. and its just mosquitoes i am seeing')


Well don't fret John. We see this kind of thing all the time. Give me just a second to check some pricing for you!
{'pest_id': '16', 'special_instructions': None, 'location': 'All throughout the yard, but mostly back patio', 'square_footage': '3000', 'property_type': 'Single Family Home', 'acres': '2.5', 'target_pests': 'mosquitoes'}


> Entering new SequentialChain chain...

> Finished chain.
Based on your needs, I would recommend our Lawnshield Mosquito Strategy with our IN2Care program. This program involves placing mosquito traps throughout your property and offers a green strategy that's perfect for a townhome like yours. With our program, you can enjoy your lawn without having to worry about mosquito bites or the anxiety that comes with these pests! Plus, our automatic billing and scheduling will make things simple and easy for you. The initial cost for this program is $168 and the recurring price is $84. Would you like me to set you up with this service plan today? 😊 

Payment L

In [36]:
print(agent.natural_history())

Agent: Hi John, how can I help you today?
Customer: Do you offer mosquito control?
Agent: Absolutely, we have our Lawnshield Mosquito Strategy that can help you enjoy your yard again! 😊 Can I ask you a few questions to understand your needs and provide you with an accurate quote?
Customer: Sure thats fine
Agent: Great, let's start with the location of the issue on your property. Where are you experiencing the most mosquito activity?
Customer: All throughout the yard but mostly the back patio
Agent: Got it, thanks for the info. Can you tell me the square footage of your property? This will help me get an idea of the area we'll need to treat.
Customer: 3000 square feet
Agent: Thanks! And what about the acreage of your property? This will help us in determining the appropriate service plan for you.
Customer: 2.5 acres
Agent: Sounds like a nice space! Now, can you please let me know the type of property you have? Is it a single-family home, townhome, apartment, condo, or commercial propert

In [37]:
print(agent.offers)

IN2Care
Initial Price: $168
Recurring Price: $84
Description: Green tick and mosquito control program involving mosquito traps that are placed throughout the property. Offers a lower amount of protection than spraying.
Covered Pests: mosquitoes
Best For: Customers who have a mosquito issue and want a green strategy or don't want spraying
Frequency: Monthly April to October
Level of Protection: Average
Benefits: Stop itchy mosquito bites; Enjoy your lawn without anxiety about mosquitoes; A feel-good green option to avoid chemicals and protect the environment;  Easy autopilot system with automatic billing and scheduling
Payment Link: <pay:Program 6>

IN2Care
Initial Price: $168
Recurring Price: $84
Description: Green tick and mosquito control program involving mosquito traps that are placed throughout the property. Offers a lower amount of protection than spraying.
Covered Pests: mosquitoes
Best For: Customers who have a mosquito issue and want a green strategy or don't want spraying
Fre

In [2]:
hist = """"Agent: Hi John, how can I help you today?
Customer: Do you offer mosquito control?
Agent: Absolutely, we have our Lawnshield Mosquito Strategy that can help you enjoy your yard again! 😊 Can I ask you a few questions to understand your needs and provide you with an accurate quote?
Customer: Sure thats fine
Agent: Great, let's start with the location of the issue on your property. Where are you experiencing the most mosquito activity?
Customer: All throughout the yard but mostly the back patio
Agent: Got it, thanks for the info. Can you tell me the square footage of your property? This will help me get an idea of the area we'll need to treat.
Customer: 3000 square feet
Agent: Thanks! And what about the acreage of your property? This will help us in determining the appropriate service plan for you.
Customer: 2.5 acres
Agent: Sounds like a nice space! Now, can you please let me know the type of property you have? Is it a single-family home, townhome, apartment, condo, or commercial property?
Customer: its a single family town home
Agent: Perfect, thanks for the information. And besides mosquitos, are you dealing with any other pests that you'd like us to address as well? 🐛"""

test_response = analyze_situation_sequential({
    "property_type": "single family home",
    "square_footage": 3000,
    "acres": 2.5,
    "target": 'mosquito',
    "conversation_history": hist
})



> Entering new SequentialChain chain...


NoIndexException: Index is not initialized

In [65]:
test_response['situation_analysis']

'Based on the conversation, the customer is experiencing mosquito activity throughout their 2.5 acre, 3000 square foot single-family townhome property. The agent also asks about other pests, implying that the focus here is on mosquito control. The practical experiences mentioned suggest that factors like yard wetness, shade, and vegetation will affect the effectiveness of treatment and pricing. \n\nOverall, this issue seems to be of Medium severity. While the customer is experiencing a problem with mosquitos, it is not a life-threatening or urgent issue. However, it is impacting their ability to enjoy their yard and may be a nuisance. Additionally, the size of the property may require a more specific treatment plan and could result in higher prices.'

In [4]:
from ChooseService import service_description_search
service_description_search.similarity_search('Based on the conversation, the customer is experiencing mosquito activity throughout their 2.5 acre, 3000 square foot single-family townhome property. The agent also asks about other pests, implying that the focus here is on mosquito control. The practical experiences mentioned suggest that factors like yard wetness, shade, and vegetation will affect the effectiveness of treatment and pricing. \n\nOverall, this issue seems to be of Medium severity. While the customer is experiencing a problem with mosquitos, it is not a life-threatening or urgent issue. However, it is impacting their ability to enjoy their yard and may be a nuisance. Additionally, the size of the property may require a more specific treatment plan and could result in higher prices.', 3)

NoIndexException: Index is not initialized

In [63]:
programs = test_response['complete_services']
#iterate over the property/value pairs in the program
for program in programs:
    for key, value in program.dict.items():
        print(key, value)

Name Program 6
Outward Facing IN2Care
Best for Customers who have a mosquito issue and want a green strategy or don't want spraying
Description Green tick and mosquito control program involving mosquito traps that are placed throughout the property. Offers a lower amount of protection than spraying.
Frequency Monthly April to October
Covered Pests mosquitoes
Not covered pests ants, spiders, rodents, millipedes, roaches, ticks
Level of Protection Average
Cost Average
Recurring Formula 75 + ((2.5-0.25)*4)
Initial Formula (75 + ((2.5-0.25)*4))*2
Billing Cycle Monthly
Benefits Stop itchy mosquito bites; Enjoy your lawn without anxiety about mosquitoes; A feel-good green option to avoid chemicals and protect the environment;  Easy autopilot system with automatic billing and scheduling
Service ID prod_Na92NNQrZFIqZM
Recurring Price 84
Initial Price 168
Payment Link https://checkout.stripe.com/c/pay/cs_test_a17rgboVnK7DrNWpDC5LxwXIrVMdqvI6bFXhsvO0cMUH6B6fckqfdI7bbg#fidkdWxOYHwnPyd1blpxYHZxWjA